In [3]:
pip install semantic-kernel

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install pymongo python-dotenv semantic-kernel pydantic flask_cors

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install nest_asyncio

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Load the API key from a file
with open("OpenAI_API.txt", "r") as file:
    openai_api_key = file.read().strip()

# Use it (example for OpenAI library)
import openai
openai.api_key = openai_api_key


In [5]:
import os
from dotenv import load_dotenv
load_dotenv()  # Load the .env file
model_id_agent1 = os.getenv("OPENAI_CHAT_MODEL3")
model_id_agent2 = os.getenv("OPENAI_CHAT_MODEL4")
api_key = os.getenv("OPENAI_API_KEY")

In [6]:
import asyncio
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import  OpenAIChatCompletion,OpenAIChatPromptExecutionSettings
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.functions import kernel_function,KernelArguments
from typing import Annotated
from pydantic import BaseModel
import pandas as pd

transactions = pd.DataFrame({
    "merchant": ["Starbucks", "Chipotle", "Walmart", "Starbucks", "Domino's"],
    "category": ["Food", "Food", "Groceries", "Food", "Food"],
    "amount": [5.50, 12.75, 80.00, 6.25, 18.00],
    "date": ["2024-03-10", "2024-03-12", "2024-03-15", "2024-03-18", "2024-03-21"]
})

# @kernel_function
# def classify_task(self, user_input: str) -> str:
#     """
#     Return either 'analysis' or 'data_retrieval' based on user intent.
#     """
    
# 🧩 Plugin class with logging
class ExpensePlugin:
    @kernel_function(description="Get total monthly food spending.")
    def get_monthly_food_expense(self) -> Annotated[str, "Returns the total spent on food last month."]:
        print("[LOG] Plugin function 'get_monthly_food_expense' was called by Analyst_Agent.")
        return "[ExpensePlugin used] Based on your data, you spent $452 on food last month."

class DatabasePlugin:

    @kernel_function(description="Get all restaurant names the user spent money on.")
    def get_restaurants(self) -> Annotated[str, "Returns a list of unique food merchants."]:
        print("[LOG] get_restaurants called.")
        food_merchants = transactions[transactions["category"] == "Food"]["merchant"].unique()
        return ", ".join(food_merchants)

    @kernel_function(description="Get total spending at a specific merchant.")
    def get_spending_by_merchant(self, merchant: Annotated[str, "The name of the merchant."]) -> Annotated[str, "Returns total spent."]:
        print(f"[LOG] get_spending_by_merchant called for: {merchant}")
        total = transactions[transactions["merchant"].str.lower() == merchant.lower()]["amount"].sum()
        return f"You spent ${total:.2f} at {merchant}."

# Structured response (optional)
class ExpenseSummary(BaseModel):
    category: str
    amount: float

thread = None
settings = OpenAIChatPromptExecutionSettings()
#settings.response_format = ExpenseSummary  # Structured output
#args = KernelArguments(settings)

#this is the analyst agent that will use tools as plugin and handle all the issues like cost summary tables, give recommendations on savings, and analyse the data as Customer sercvice agent requested
Analyst_Agent = ChatCompletionAgent(
        service=OpenAIChatCompletion(ai_model_id=model_id_agent2, api_key=api_key),
        name="Analyst_Agent",
        instructions= "You are an invisible backend analyst.. Evaluate user requests using appropriate tools. "\
            "Always begin with: '[Analyst_Agent activated]'. "\
            "If you can use plugins to get the data you need and complete the task."\
            "You are not allowed to use any other plugins or tools which not in the plugins that provided to you. "
            "If the request is not solvable with plugins, reply with: '[Forwarding back to Customer_Service_Agent]'."
            "Otherwise, provide the completed analysis directly as part of your response.",
        plugins=[ExpensePlugin()], # 所有function放在这里
        arguments=KernelArguments(settings)
    )

Database_Agent = ChatCompletionAgent(
    service=OpenAIChatCompletion(ai_model_id=model_id_agent2, api_key=api_key),
    name="Database_Agent",
    instructions=(
        "You are a backend data retrieval agent. "
        "You do NOT interact with the user directly. "
        "Only the Customer_Service_Agent can talk to the user. "
        "If a request lacks information (e.g., merchant name, date, category), ask the Customer_Service_Agent to gather the missing info. "
        "Be concise, and return data or ask only for clarification needed to complete the task."
    ),
    plugins=[DatabasePlugin()]
)

# This is the main agent that commnunicate with user
Customer_Service_Agent = ChatCompletionAgent(
    service=OpenAIChatCompletion(ai_model_id=model_id_agent1, api_key=api_key),
    name="Customer_Service_Agent",
    instructions=(
        "You are the only agent who talks to the user. "
        "You use internal agents to fulfill requests: \n"
        "- Use **Analyst_Agent** for any task that involves analysis, summaries, trends, total spending calculations, charts, or recommendations. \n"
        "- Use **Database_Agent** only for direct fact-based retrieval, such as 'what restaurants did I go to', 'when did I visit Starbucks', or 'how many times did I shop at Walmart'.\n\n"
        "If a request lacks information (like merchant or date), ask Database_Agent to clarify — then ask the user. "
        "Always integrate responses and present them in your own voice. Do not reveal internal agents."
    ),
    plugins=[Analyst_Agent, Database_Agent]
)

# Customer_Service_Agent = ChatCompletionAgent(
#         service=OpenAIChatCompletion(ai_model_id=model_id_agent1, api_key=api_key),
#         name="Customer_Service_Agent",
#         instructions="You are a customer service assistant. Handle general inquiries. "\
#         "If the user asks for specific calculations like food spending, forward the request to Analyst_Agent and wait for a result. "\
#         "Prefix this with '[Forwarding to Analyst_Agent]'. After receiving the result, prefix it with '[Response from Analyst_Agent]' "\
#         "and respond fully. After every response, ask the user if they have another question." ,
#         plugins=[Analyst_Agent, Database_Agent],  # Add the Analyst_Agent as a plugin
#     )

thread: None


async def main():
    print("[LOG] User sends message to Customer_Service_Agent...")
    #user_message = "Which restaurants did I spend on this month?"
    user_message = "Can you help me to analyze how much I spent in total last month?"

    print("[LOG] User asked:", user_message)

    response = await Customer_Service_Agent.get_response(messages=user_message)
    print("[LOG] Final response from Customer_Service_Agent:")
    print(response.content)

await main()
# Output:
# Language's essence,
# Semantic threads intertwine,
# Meaning's core revealed.

[LOG] User sends message to Customer_Service_Agent...
[LOG] User asked: Can you help me to analyze how much I spent in total last month?
[LOG] Plugin function 'get_monthly_food_expense' was called by Analyst_Agent.
[LOG] Final response from Customer_Service_Agent:
You spent a total of $452 on food last month. If you need more specific details or analysis, feel free to ask!
